Question2-Part2
-Read only the json files which contain the key called “restaurants”
-Each (or most of the json files) contain a key called “open” which contains the details of the operation (timings) of the restaurants. For each json file, read the timings of the restaurants.
-Data of the operation timings of the restaurants is present for each day of the week. I want you to extract each of this data and write it in an excel sheet.

In [231]:
import xlsxwriter
import json
import glob
import xlrd
import csv

In [123]:
path_local = '/Users/lisha/Desktop/INFO7374_PythonIntro_Homework/Assignment2/Question2/'
list_jsonfiles = glob.glob(path_local + 'Data/*.json')
print(list_jsonfiles[:2])

['/Users/lisha/Desktop/INFO7374_PythonIntro_Homework/Assignment2/Question2/Data/1-abercrombie-lane-sydney.json', '/Users/lisha/Desktop/INFO7374_PythonIntro_Homework/Assignment2/Question2/Data/1-chicken-rice-and-seafood-houston.json']


In [124]:
data_dic = {}
for file in list_jsonfiles:
    with open(file) as data_file:
        data_dic[file] = json.load(data_file)

In [125]:
path_test = '/Users/lisha/Desktop/INFO7374_PythonIntro_Homework/Assignment2/Question2/Data/204-north-kitchen-and-cocktails-charlotte.json'
print(data_dic[path_test])

{'price': '$$', 'rating': 3.5, 'coordinates': {'longitude': -80.8415232515553, 'latitude': 35.227895120146}, 'is_claimed': True, 'hours': [{'is_open_now': False, 'open': [{'is_overnight': False, 'end': '0000', 'start': '1100', 'day': 0}, {'is_overnight': False, 'end': '0000', 'start': '1100', 'day': 1}, {'is_overnight': True, 'end': '0200', 'start': '1100', 'day': 2}, {'is_overnight': True, 'end': '0200', 'start': '1100', 'day': 3}, {'is_overnight': True, 'end': '0200', 'start': '1100', 'day': 4}, {'is_overnight': True, 'end': '0200', 'start': '1100', 'day': 5}, {'is_overnight': False, 'end': '0000', 'start': '1100', 'day': 6}], 'hours_type': 'REGULAR'}], 'photos': ['https://s3-media1.fl.yelpcdn.com/bphoto/IjsmSX4vao-5VSkZtQOXUw/o.jpg', 'https://s3-media3.fl.yelpcdn.com/bphoto/Rc8jqWTs7CajODSv7Mb0Og/o.jpg', 'https://s3-media3.fl.yelpcdn.com/bphoto/S55GvNFII9ACX1nnIxs36A/o.jpg'], 'review_count': 61, 'id': '204-north-kitchen-and-cocktails-charlotte', 'term': 'restaurants', 'url': 'https:

In [182]:
restaurant_dic = {}
for key in data_dic.keys(): 
    item = {}
    item = data_dic[key]
    if item['term']=='restaurants':        
        hours = []
        if 'hours' in item.keys():
            for list_hours in item['hours'][0]['open']:
                i=0
                start_hour = list_hours['start'][:2]
                start_min = list_hours['start'][-2:]
                end_hour = list_hours['end'][:2]
                end_min = list_hours['end'][-2:]
                hours.append([i,start_hour,start_min,end_hour,end_min])
                i+=1
            restaurant_dic[item['name']] = {'city': item['location']['city'], 'country':item['location']['country'], 'hours':hours}
        else:
            restaurant_dic[item['name']] = {'city': item['location']['city'], 'country':item['location']['country'], 'hours':'none'}

In [229]:
print(restaurant_dic['204 North Kitchen & Cocktails'])

{'city': 'Charlotte', 'country': 'US', 'hours': [[0, '11', '00', '00', '00'], [0, '11', '00', '00', '00'], [0, '11', '00', '02', '00'], [0, '11', '00', '02', '00'], [0, '11', '00', '02', '00'], [0, '11', '00', '02', '00'], [0, '11', '00', '00', '00']]}


In [251]:
workbook = xlsxwriter.Workbook('RestaurantHours.xlsx')
worksheet = workbook.add_worksheet('sheet1')
bold = workbook.add_format({'bold': 1})
worksheet.write('A1', 'Name of restaurant', bold)
worksheet.write('B1', 'City', bold)
worksheet.write('C1', 'Country Code', bold)
worksheet.write('D1', 'Day of week', bold)
worksheet.write('E1', 'Start Time Hour', bold)
worksheet.write('F1', 'Start Time Minute', bold)
worksheet.write('G1', 'End time Hour', bold)
worksheet.write('H1', 'End Time Minute', bold)

i=1
for key in restaurant_dic.keys():
    restaurant = restaurant_dic[key]
    if 'none' not in restaurant['hours']:
        for j in range(len(restaurant['hours'])):
            worksheet.write(i+j ,0, key)
            worksheet.write(i+j, 1, restaurant['city'])
            worksheet.write(i+j, 2, restaurant['country'])
            worksheet.write(i+j, 3, restaurant['hours'][j][0])
            worksheet.write(i+j, 4, restaurant['hours'][j][1])
            worksheet.write(i+j, 5, restaurant['hours'][j][2])
            worksheet.write(i+j, 6, restaurant['hours'][j][3])
            worksheet.write(i+j, 7, restaurant['hours'][j][4])
    else:
        for j in range(len(restaurant['hours'])):
            worksheet.write(i+j ,0, key)
            worksheet.write(i+j, 1, restaurant['city'])
            worksheet.write(i+j, 2, restaurant['country'])
            worksheet.write(i+j, 3, 'none')
            worksheet.write(i+j, 4, 'none')
            worksheet.write(i+j, 5, 'none')
            worksheet.write(i+j, 6, 'none')
            worksheet.write(i+j, 7, 'none')
    i = i+j+1
workbook.close()

In [255]:
wb = xlrd.open_workbook(path_local + 'RestaurantHours.xlsx')
sh = wb.sheet_by_name('sheet1')
your_csv_file = open(path_local + 'RestaurantHours.csv','w')
wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)
for rownum in range(sh.nrows):
    wr.writerow(sh.row_values(rownum))
your_csv_file.close()